<a href="https://colab.research.google.com/github/BigtoC/ANN-Diabetes-Classification/blob/master/ANN_Diabetes_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CS4486 Artificial Intelligence Project 2 
You are required to implement an artificial neural network (ANN) for solving  
a diabetes classification problem using the training method of genetic algorithm.

## Import Libraries

In [0]:
!pip3 install ann_visualizer

In [0]:
from keras.layers import Dense, Dropout, Activation
from keras.models import Sequential
from keras.utils import plot_model
import tensorflow as tf
import pandas as pd
import numpy as np
import datetime
import os
from ann_visualizer.visualize import ann_viz

from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier

np.random.seed(20204486)

In [0]:
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)

Check the GPU

In [16]:
!nvidia-smi

Sun May  3 13:36:11 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.64.00    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   69C    P0    32W /  70W |    317MiB / 15079MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

## Load dataset
The first column of the diabetes data file is the bias input,  
and the second to the ninth columns are the input features.  
The tenth and eleventh columns of the diabetes data file are the output values.  
A one in the tenth column would represent diabetes positive.  
There are a total of 768 patterns in the dataset.  
576 are used as training pattern and 192 are used as test pattern. 

### Read data from txt file

In [17]:
from google.colab import drive
drive.mount('/content/drive')
base_path = '/content/drive/My Drive/Colab Notebooks/CS4486/ANN-Diabetes-Classification'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [18]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

# Clear any logs from previous runs
!rm -rf ./logs/

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


#### Read data and store in a dateframe (for showing the dataset)

In [19]:
data_path = f'{base_path}/diabetes.txt'
diabetes_df = pd.read_csv(data_path, sep="\t", header=None)
diabetes_df

,0,1,2,3,4,5,6,7,8,9,10
0,1,0.176471,0.605,0.426230,0.00,0.000000,0.536513,0.020922,0.066667,1,0
1,1,0.352941,0.720,0.590164,0.27,0.269504,0.505216,0.075576,0.316667,0,1
2,1,0.117647,0.875,0.721311,0.00,0.000000,0.341282,0.105892,0.016667,0,1
3,1,0.705882,0.605,0.639344,0.17,0.000000,0.394933,0.077284,0.683333,0,1
4,1,0.117647,0.535,0.606557,0.30,0.118203,0.500745,0.139197,0.033333,0,1
...,...,...,...,...,...,...,...,...,...,...,...
763,1,0.117647,0.280,0.459016,0.28,0.053191,0.360656,0.108454,0.016667,0,1
764,1,0.117647,0.545,0.754098,0.00,0.000000,0.636364,0.327498,0.550000,0,1
765,1,0.176471,0.450,0.639344,0.00,0.000000,0.636364,0.205380,0.000000,0,1
766,1,0.058824,0.510,0.606557,0.00,0.000000,0.588674,0.091802,0.350000,1,0


#### Read data and store as a nparray (for calculation)

In [0]:
diabetes_np = np.loadtxt(data_path, delimiter='\t')

### Split to train and test

In [47]:
# Shuffle the array
np.random.shuffle(diabetes_np)

# Split by 80/20
train_count = int(len(diabetes_np) * 0.8)

train_diabetes, test_diabetes = diabetes_np[:train_count,:], diabetes_np[train_count:,:]

print(f"There are {len(train_diabetes)} rows for training set, and {len(test_diabetes)} for testing set")

There are 614 rows for training set, and 154 for testing set


#### Split features and targets

In [0]:
def split_features_targets(an_array): 
    features = an_array[:, 1 : 9]
    targets = an_array[:, 9: 11]
    return features, targets

In [0]:
train_features, train_targets = split_features_targets(train_diabetes)
test_features, test_targets = split_features_targets(test_diabetes)

In [134]:
train_targets

array([[1., 0.],
       [1., 0.],
       [0., 1.],
       ...,
       [0., 1.],
       [0., 1.],
       [1., 0.]])

## Model

### Create the model

In [0]:
def create_model():
    model = Sequential()
    model.add(Dense(32, input_dim=8, activation='relu'))
    
    model.add(Dense(16, activation='relu'))
    
    model.add(Dense(2, activation='sigmoid'))

    model.summary()
    return model

### Compile and train the model

In [157]:
# Get the model
model = create_model()

# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Load tensorboard
log_dir = f"logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

# Train the model
model.fit(x=train_features, y=train_targets, 
          epochs=1000, batch_size=500, validation_split=0.2
          )

Model: "sequential_15"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_45 (Dense)             (None, 32)                288       
_________________________________________________________________
dense_46 (Dense)             (None, 16)                528       
_________________________________________________________________
dense_47 (Dense)             (None, 16)                272       
_________________________________________________________________
dense_48 (Dense)             (None, 2)                 34        
Total params: 1,122
Trainable params: 1,122
Non-trainable params: 0
_________________________________________________________________
Train on 491 samples, validate on 123 samples
Epoch 1/1000
491/491 [==============================] - 0s 244us/step - loss: 0.6880 - accuracy: 0.5580 - val_loss: 0.6872 - val_accuracy: 0.5935
Epoch 2/1000
491/491 [==============================] - 0

In [0]:
ann_viz(model, view=True, title='ann-model.png')

### Evaluate the model

In [0]:
def print_evaluations(which: str):
    the_features, the_targets = None, None
    metrics_num = len(model.metrics_names)

    if which is 'Training':
        the_features = train_features
        the_targets = train_targets
    elif which is 'Testing':
        the_features = test_features
        the_targets = test_targets
    else:
        print("Attribute error! Please input 'Training' or 'Testing'!" )
    
    print(f"Evaluating moedl with {which.lower()} dateset")

    the_scores = model.evaluate(x=the_features, y=the_targets, batch_size=320)
    
    for i in range(metrics_num):
        print(f"{which} {model.metrics_names[i]}: {round(the_scores[i], 3)}")
    print()


In [158]:
# Evaluate Training 
print_evaluations("Training")

# Evaluate Testing 
print_evaluations("Testing")

Evaluating moedl with training dateset
614/614 [==============================] - 0s 21us/step
Training loss: 0.351
Training accuracy: 0.86

Evaluating moedl with testing dateset
154/154 [==============================] - 0s 21us/step
Testing loss: 0.627
Testing accuracy: 0.766



In [0]:
%tensorboard --logdir logs/fit

## Compare with Random Forest

In [146]:
rf = RandomForestClassifier(
    criterion='entropy',
    n_estimators=100,
    max_depth=20, 
    )
rf.fit(train_features, train_targets)
rf_roc_auc = metrics.roc_auc_score(test_targets, rf.predict(test_features))
print(f"Random Forest AUC = {round(rf_roc_auc, 2)}")
print(metrics.classification_report(test_targets, rf.predict(test_features)))

Random Forest AUC = 0.73
              precision    recall  f1-score   support

           0       0.62      0.62      0.62        48
           1       0.83      0.83      0.83       106

   micro avg       0.77      0.77      0.77       154
   macro avg       0.73      0.73      0.73       154
weighted avg       0.77      0.77      0.77       154
 samples avg       0.77      0.77      0.77       154

